In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from utils.categorize_brand import categorize_brand

df = pd.read_csv("../data/sales_ads_train.csv")

 # buduje listę ilośći aut w widełkach co 50k zeby zobaczyć gdzie nalezy oczyścić dane
bins = list(range(0, int(df["Cena"].max()) + 50000, 50000)) 
labels = [f"{bins[i]}-{bins[i+1]}" for i in range(len(bins)-1)]
df["Widełki_cenowe"] = pd.cut(df["Cena"], bins=bins, labels=labels, include_lowest=True)
widełki_count = df["Widełki_cenowe"].value_counts().sort_index()

# ucinam ceny do 1000 (38 ofert) i od 2 mln praktycznie brak ofert
df_filtered = df[(df["Cena"] >= 1000) & (df["Cena"] <= 2000000)]

# usuwam zbędne kolumny - "Widełki_cenowe", "Data_publikacji_oferty", "ID"
df_filtered = df_filtered.drop(columns=["Widełki_cenowe", "Data_publikacji_oferty", "ID"])

# usuwam wieksze w których waluta != PLN oraz usuwam zbędną kolumnę
df_filtered = df_filtered[df_filtered["Waluta"] == "PLN"]
df_filtered = df_filtered.drop(columns=["Waluta"])

# usuwamy kolumny które mają duzo braków a potencjalnie mały wpływ na cenę
df_filtered = df_filtered.drop(columns=["Wersja_pojazdu", "Generacja_pojazdu", "Emisja_CO2","Data_pierwszej_rejestracji"])

# Pierwszy właściciel ma tylko 2 wartości - [nan, 'Yes'] więc zamieniamy je na 0 i 1
df_filtered["Pierwszy_wlasciciel"] = df_filtered["Pierwszy_wlasciciel"].notna().astype(int)
# df_filtered.info()

# Kraj pochodzenia podnosi cenę tylko jeśli to jest Polska więc zamieniamy na 1 dla Polski i 0 dla reszty świata
df_filtered["Kraj_pochodzenia"] = (df_filtered["Kraj_pochodzenia"] == "Poland").astype(int)

# Stan jest ['Used', 'New', nan] więc nan traktujemy jako used
df_filtered["Stan"] = (df_filtered["Stan"] == "New").astype(int)

# Grupujemy marki na premium, srednie i budzetowe
df_filtered["Segment_marki"] = df_filtered["Marka_pojazdu"].apply(categorize_brand)
# Przypisujemy segmenty marek
df_filtered["Segment_marki"] = df_filtered["Marka_pojazdu"].apply(categorize_brand)

# df_filtered = pd.get_dummies(df_filtered, columns=["Segment_marki"], dtype=int)
# df_filtered.info()
# df_filtered['Segment_marki'].value_counts()

Segment_marki
Średnie      59632
Budżetowe    41701
Premium      30491
Name: count, dtype: int64